In [ ]:
!git clone https://github.com/airtlab/ConvLSTM4AD

#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install simpleitk
import SimpleITK as sitk

def resample_img(itk_image, out_spacing=[1, 1, 1], is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)
    return resample.Execute(itk_image)

def preprocess(img, out_shape=None):
    if out_shape is not None:
        img = resize(img, out_shape, mode='constant')
    
    mean = img.mean() 
    std = img.std() 
    return (img - mean) / std

In [ ]:
import os
import nibabel as nib
import numpy as np

img_path = "/content/ConvLSTM4AD/Merged_dataset/Dataset"
img_height, img_width = 256, 256 
seq_len = 5

X = [] # an empty list ‘X’ is generated
Y = [] # an empty list ‘Y’ is generated

dir = img_path # the content of the string ‘img_path’ is stored in a new string ‘dir’ to simplify its subsequent usage
classes = ["AD","CN"] # two classes are generated 
classes_list = os.listdir(dir) # a list containing the names of the elements inside ‘dir’ is generated
classes_list.sort() # the list is sorted to have AD as first element and CN as second one

full_path_AD = os.path.join(dir, classes_list[0]) # the path of the AD directory is obtained joining ‘dir’ with the first element of ‘classes_list’
samples_AD = os.listdir(full_path_AD) # a list containing the names of the elements inside the AD directory is generated 
samples_AD.sort() # the sample names are sorted alphabetically 
for a in samples_AD: 
  b = os.path.join(full_path_AD, a) # the file path is stored joining ‘full_path_AD’ with the current sample name ‘a’ 
  my_img = nib.load(b) 
  nii_data = my_img.get_fdata() 
  my_img.uncache()
  del my_img
  temp = [] # a new empty list ‘temp’ is generated
  for i in range(0,5): 
    temp.append(preprocess(nii_data[:,:,i]))
  X.append(temp) # once all slices have been appended, ‘temp’ is appended to the ‘X’ list
  y = [0]*len(classes_list) # a 1x2 array ‘y’ filled with zeros is generated
  y[1] = 1 # 1 is assigned to the second column of the array ([0,1] is the AD label)
  Y.append(y) # the ‘y’ array is appended to the ‘Y’ list

full_path_CN = os.path.join(dir, classes_list[1]) # the path of the CN directory is obtained joining ‘dir’ with the second element of ‘classes_list’
samples_CN = os.listdir(full_path_CN) 
samples_CN.sort()
for a in samples_CN: 
  b = os.path.join(full_path_CN, a)
  my_img = nib.load(b) 
  nii_data = my_img.get_fdata() 
  my_img.uncache()
  del my_img
  temp = []
  for i in range(0,5): 
    temp.append(preprocess(nii_data[:,:,i]))
  X.append(temp)
  y = [0]*len(classes_list)
  y[0] = 1 # 1 is assigned to the first column of the array ([1,0] is the CN label)
  Y.append(y)  

In [4]:
X = np.asarray(X) # ‘X’ is turned into a numpy array
Y = np.asarray(Y) # the same for ‘Y’

X = X[...,np.newaxis] # a new axis is added to ‘X’ to match the shape requested by the training function
print(X.shape)

(427, 5, 256, 256, 1)


In [5]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split

seed = 0 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, 
                                                    shuffle=True, random_state=seed) # split in train (train+val) and test: 80% of the total for train+val and 20% for test 
                                                                                     # ‘shuffle’ shuffles all data before splitting
                                                                                     # ‘random_state’ controls the shuffling: each integer is related to a specific shuffle

print("Number of training samples: " + str(len(X_train)))
print("Number of test samples: " + str(len(X_test)))

del X
del Y

Number of training samples: 341
Number of test samples: 86


In [7]:
#!pip show tensorflow
#!pip install --upgrade tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential()

model.add(layers.ConvLSTM2D(filters=32, 
                            kernel_size=(3,3), 
                            return_sequences=False, 
                            data_format="channels_last", 
                            input_shape=(seq_len,img_height,img_width, 1)))
model.add(layers.Dropout(0.5)) 
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(2, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 254, 254, 32)      38144     
_________________________________________________________________
dropout (Dropout)            (None, 254, 254, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 2064512)           0         
_________________________________________________________________
dense (Dense)                (None, 256)               528515328 
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 528,553,986
Trainable params: 528,553,986
Non-trainable params: 0
__________________________________________

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001) 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

earlystop = tf.keras.callbacks.EarlyStopping(patience=7)
callbacks = [earlystop]

history = model.fit(x=X_train, y=Y_train, epochs=40, batch_size=4, shuffle=True, validation_split=0.2, callbacks=callbacks)

Epoch 1/40
68/68 [==============================] - 140s 2s/step - loss: 2.7332 - accuracy: 0.6459 - val_loss: 0.4357 - val_accuracy: 0.8116
Epoch 2/40
68/68 [==============================] - 132s 2s/step - loss: 0.5120 - accuracy: 0.7520 - val_loss: 0.4318 - val_accuracy: 0.7971
Epoch 3/40
68/68 [==============================] - 131s 2s/step - loss: 0.3696 - accuracy: 0.8430 - val_loss: 0.3744 - val_accuracy: 0.7826
Epoch 4/40
68/68 [==============================] - 130s 2s/step - loss: 0.3333 - accuracy: 0.8506 - val_loss: 0.3744 - val_accuracy: 0.7826
Epoch 5/40
68/68 [==============================] - 129s 2s/step - loss: 0.2934 - accuracy: 0.8794 - val_loss: 0.3287 - val_accuracy: 0.7971
Epoch 6/40
68/68 [==============================] - 128s 2s/step - loss: 0.3438 - accuracy: 0.8411 - val_loss: 0.3377 - val_accuracy: 0.8261
Epoch 7/40
68/68 [==============================] - 126s 2s/step - loss: 0.3007 - accuracy: 0.8690 - val_loss: 0.3430 - val_accuracy: 0.7826
Epoch 8/40
68